- TODO: Parallelize sharp wave property computation

# Imports and definitions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [4]:
from ecephys.data import paths, channel_groups
from ecephys.sglx_utils import load_timeseries, load_multifile_timeseries
from ecephys.signal.csd import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves_by_value, detect_sharp_waves_by_zscore, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.utils import load_df_h5, store_df_h5, zscore_to_value
import ecephys.plot as eplt

In [9]:
def get_spw_detection_parameters(subject, detection_threshold_zscore=2.5, boundary_threshold_zscore=1):
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    hpc_chans = channel_groups.hippocampus[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition="recovery-sleep-2h", ext="lf.bin")
    params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")
    
    if len(bin_paths) > 1:
        (time, hpc_lfps, fs) = load_multifile_timeseries(bin_paths, hpc_chans)
    else:
        (time, hpc_lfps, fs) = load_timeseries(bin_paths[0], hpc_chans)
    
    gdx = intersite_distance = 0.020
    k = get_kcsd(
        hpc_lfps, intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
    )
    
    hpc_csd = k.values("CSD")
    sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]
    combined_csd = np.sum(-sr_csd.T, axis=1)

    detection_threshold = zscore_to_value(combined_csd, detection_threshold_zscore)
    boundary_threshold = zscore_to_value(combined_csd, boundary_threshold_zscore)
    
    metadata = dict(
        csd_chans=hpc_chans.tolist(),
        detection_chans=sr_chans,
        electrode_positions=k.ele_pos.tolist(),
        intersite_distance=intersite_distance,
        gdx=k.gdx,
        lambd=k.lambd,
        R=k.R,
        detect_states=["all"],
        detection_threshold_zscore=detection_threshold_zscore,
        boundary_threshold_zscore=boundary_threshold_zscore,
        detection_threshold=detection_threshold,
        boundary_threshold=boundary_threshold,
        minimum_duration=0.005,
        params_source_files=[str(path) for path in bin_paths] 
    )
    
    params_path.parent.mkdir(parents=True, exist_ok=True)
    with open(params_path, "x") as params_file:
        json.dump(metadata, params_file, indent=4)

In [6]:
def run_spw_detection_pipeline_on_file(bin_path, spw_path, params_path, sr_chans, hpc_chans):
    (time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans)

    with open(params_path) as params_file:
        params = json.load(params_file)

    intersite_distance = params["intersite_distance"]
    k = get_kcsd(
        hpc_lfps,
        intersite_distance=params["intersite_distance"],
        gdx=params["gdx"],
        lambd=params["lambd"],
        R_init=params["R"],
        do_lcurve=False,
    )

    hpc_csd = k.values("CSD")
    sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

    spws = detect_sharp_waves_by_value(
        time,
        sr_csd,
        params["detection_threshold"],
        params["boundary_threshold"],
        params["minimum_duration"],
    )

    spws["duration"] = get_durations(spws)
    spws["midpoint"] = get_midpoints(spws)
    spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (
        1e-6
    )  # Scale to mA/mm
    spws["sink_integral"] = (
        get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3)
    )  # Scale to mA * ms

    metadata = dict(
        csd_chans=hpc_chans,
        detection_chans=sr_chans,
        electrode_positions=k.ele_pos,
        intersite_distance=intersite_distance,
        gdx=k.gdx,
        lambd=k.lambd,
        R=k.R,
        detect_states=["all"],
    )
    metadata.update(spws.attrs)

    store_df_h5(spw_path, spws, **metadata)

In [7]:
def run_spw_detection_pipeline_on_condition(subject, condition):
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    hpc_chans = channel_groups.hippocampus[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="lf.bin")
    spw_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="spws.h5")
    params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")

    for bin_path, spw_path in zip(bin_paths, spw_paths):
        run_spw_detection_pipeline_on_file(bin_path, spw_path, params_path, sr_chans, hpc_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

### Segundo

In [16]:
get_spw_detection_parameters(subject="Segundo", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 18000001
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-11
max lambda 0.0133
min lambda 1e-11
max lambda 0.0133
l-curve (all lambda):  0.23
Best lambda and R =  0.00013089207687291224 ,  0.23


In [ ]:
run_spw_detection_pipeline_on_condition(subject="Segundo", condition="extended-wake")

In [12]:
run_spw_detection_pipeline_on_condition(subject="Segundo", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000001
18:03:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t23.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
18:19:47: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t24.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
18:34:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t25.imec0.lf.bin


### Valentino

In [19]:
get_spw_detection_parameters(subject="Valentino", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 18000000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0119
min lambda 1e-12
max lambda 0.0119
l-curve (all lambda):  0.23
Best lambda and R =  0.00105886675336374 ,  0.23


In [ ]:
run_spw_detection_pipeline_on_condition(subject="Valentino", condition="extended-wake")

In [11]:
run_spw_detection_pipeline_on_condition(subject="Valentino", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000000
17:14:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
17:25:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
17:43:11: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t5.imec0.lf.bin


### Doppio

In [ ]:
get_spw_detection_parameters(subject="Doppio", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

In [ ]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="extended-wake")

In [10]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000019
17:29:26: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
17:38:31: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
17:47:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t5.imec0.lf.bin


### Alessandro

In [18]:
get_spw_detection_parameters(subject="Alessandro", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
You are loading multifile SGLX data without xarray.
 Are you sure you want to do this? Please see documentation.
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  3.711471263028718e-05 ,  0.23


In [31]:
run_spw_detection_pipeline_on_condition(subject="Alessandro", condition="extended-wake")

nChan: 385, nFileSamp: 9000051
16:17:22: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:20:02: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 8127109
16:22:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 8277840
16:24:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD2_g0/8-25-2020_SD2_g0_imec0/8-25-2020_SD2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:27:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:31:12: Finished /Volumes/neuropixel_archive/Data/chronic/

In [13]:
run_spw_detection_pipeline_on_condition(subject="Alessandro", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 9000052
12:34:40: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:41:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000051
12:46:54: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:50:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t5.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:53:13: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t6.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:55:58: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25

### Eugene

In [34]:
get_spw_detection_parameters(subject="Eugene", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
You are loading multifile SGLX data without xarray.
 Are you sure you want to do this? Please see documentation.
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-11
max lambda 0.0140
min lambda 1e-11
max lambda 0.0140
l-curve (all lambda):  0.23
Best lambda and R =  0.001404472887665455 ,  0.23


In [35]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="extended-wake")

nChan: 385, nFileSamp: 800217
16:55:56: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs_g0/9.24.2020_SD_24hs_g0_imec0/9.24.2020_SD_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2877424
16:56:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs1_g0/9.24.2020_SD_24hs1_g0_imec0/9.24.2020_SD_24hs1_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2430722
16:56:59: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs2_g0/9.24.2020_SD_24hs2_g0_imec0/9.24.2020_SD_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 5735230
16:58:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs3_g0/9.24.2020_SD_24hs3_g0_imec0/9.24.2020_SD_24hs3_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
17:01:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs3_g1/9.24.2020_SD_24hs3_g1_imec0/9.24.2020_SD_24hs3_g1_t0.imec0.lf.bin
nChan: 385, nFileSa

In [14]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 9000026
13:09:44: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
13:12:33: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
13:16:04: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
13:18:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
13:21:14: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t4.imec0.lf.bin


KeyError: 'fileSizeBytes'

# Run pipeline piecemeal

In [7]:
SUBJECT = "Doppio"
CONDITION = "BSL-0+2"

## Load the data

In [8]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
hpc_chans = channel_groups.hippocampus[SUBJECT]
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")
params_path = paths.get_datapath(subject=SUBJECT, condition="REC-30+18", data="sharp_wave_detection_params.json")

In [9]:
(time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans)

nChan: 385, nFileSamp: 18000019


### Explore LFPs (optional)

In [ ]:
eplt.lfp_explorer(time, hpc_lfps, chan_labels=hpc_chans)

## Detect sharp waves

### If we need to determine detection parameters

In [8]:
intersite_distance = 0.020
k = get_kcsd(hpc_lfps, intersite_distance=intersite_distance, gdx=0.020, do_lcurve=True)
hpc_csd = k.values('CSD')
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  0.0003822395851068327 ,  0.23


In [32]:
spws = detect_sharp_waves_by_zscore(time, sr_csd)

### If we are using detection parameters obtained elsewhere

In [10]:
with open(params_path) as params_file:
    params = json.load(params_file)
intersite_distance = params['intersite_distance']

In [11]:
k = get_kcsd(hpc_lfps, intersite_distance=params['intersite_distance'], gdx=params['gdx'], lambd=params['lambd'], R_init=params['R'], do_lcurve=False)
hpc_csd = k.values('CSD')
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

In [12]:
spws = detect_sharp_waves_by_value(time, sr_csd, params['detection_threshold'], params['boundary_threshold'], params['minimum_duration'])

## Compute SPW properties

In [13]:
spws["duration"] = get_durations(spws)
spws["midpoint"] = get_midpoints(spws)

In [14]:
spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (1e-6) # Scale to mA/mm

In [15]:
spws["sink_integral"] = get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3) # Scale to mA * ms

## Export results

In [16]:
metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    electrode_positions=k.ele_pos, 
    intersite_distance=intersite_distance,
    gdx=k.gdx,
    lambd = k.lambd,
    R = k.R,
    detect_states=["all"],
)
metadata.update(spws.attrs)
spws_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="sharp_waves.h5")
store_df_h5(spws_path, spws, **metadata)

## If necessary, create params file

In [85]:
metadata.update({'params_source_file': str(bin_path)})
metadata['csd_chans'] = metadata['csd_chans'].tolist()
metadata['electrode_positions'] = metadata['electrode_positions'].tolist()
with open(params_path, 'x') as params_file:
    json.dump(metadata, params_file, indent=4)